In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargo Dataset principal

In [2]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [3]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'NYC_4') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(random_state=107,
                                        max_depth=10,
                                        min_samples_leaf=2,
                                        min_samples_split=5,
                                        n_estimators=50),
        lags=7
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicciones generadas para el producto: {producto}")


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [4]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,22,28.0,28.0
1,HOME_&_GARDEN_1_027,98,53.0,34.0
2,HOME_&_GARDEN_1_053,38,38.0,30.0
3,HOME_&_GARDEN_1_140,26,23.0,21.0
4,HOME_&_GARDEN_1_177,27,39.0,38.0
...,...,...,...,...
95,SUPERMARKET_3_499,158,122.0,121.0
96,ACCESORIES_1_158,23,34.0,32.0
97,SUPERMARKET_3_282,95,218.0,261.0
98,ACCESORIES_1_354,38,70.0,68.0


In [5]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

11764
12920.0
-1156.0


In [4]:
tienda =  pd.DataFrame(resultados)
file_name = "NYC_4_ventas.xlsx"
tienda.to_excel(file_name)

In [7]:
pd.DataFrame(resultados) #7

,producto,prediccion,test
0,ACCESORIES_1_108,"[0.7212777777777778, 1.2153492063492064, 0.537...","[0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 2, 1, 1, 0, ..."
1,HOME_&_GARDEN_1_027,"[1.6670238095238092, 2.0145000000000004, 1.926...","[6, 4, 3, 5, 1, 6, 3, 6, 2, 6, 2, 4, 2, 2, 5, ..."
2,HOME_&_GARDEN_1_053,"[1.8610988455988455, 2.641821428571429, 1.8915...","[0, 1, 0, 0, 1, 0, 1, 1, 3, 2, 0, 0, 1, 0, 1, ..."
3,HOME_&_GARDEN_1_140,"[1.0151944444444447, 1.2347261904761904, 1.095...","[0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, ..."
4,HOME_&_GARDEN_1_177,"[1.2206706349206353, 2.014960317460318, 1.3451...","[2, 5, 2, 1, 2, 2, 3, 1, 4, 1, 3, 1, 0, 0, 0, ..."
...,...,...,...
95,SUPERMARKET_3_499,"[4.8480912698412695, 4.843930375180377, 5.4851...","[6, 0, 0, 0, 6, 6, 3, 9, 15, 12, 2, 0, 7, 3, 2..."
96,ACCESORIES_1_158,"[1.0971230158730156, 1.8468571428571425, 1.165...","[1, 1, 0, 0, 0, 1, 2, 1, 2, 0, 0, 0, 0, 1, 2, ..."
97,SUPERMARKET_3_282,"[1.0208333333333335, 1.9025912698412697, 3.658...","[0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 0, 0, 2, 4, 8, ..."
98,ACCESORIES_1_354,"[2.5930915750915746, 2.0228928571428573, 3.450...","[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, ..."


In [9]:

pd.DataFrame(resultados) #7

,producto,prediccion,test
0,ACCESORIES_1_108,"[0.6784285714285716, 0.8413571428571429, 0.561...","[0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 2, 1, 1, 0, ..."
1,HOME_&_GARDEN_1_027,"[2.777111111111111, 2.0447301587301596, 1.7578...","[6, 4, 3, 5, 1, 6, 3, 6, 2, 6, 2, 4, 2, 2, 5, ..."
2,HOME_&_GARDEN_1_053,"[1.9391150793650795, 2.2443412698412697, 1.677...","[0, 1, 0, 0, 1, 0, 1, 1, 3, 2, 0, 0, 1, 0, 1, ..."
3,HOME_&_GARDEN_1_140,"[0.7773134920634921, 1.3915277777777786, 0.813...","[0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, ..."
4,HOME_&_GARDEN_1_177,"[1.2085833333333336, 1.2651370851370851, 1.548...","[2, 5, 2, 1, 2, 2, 3, 1, 4, 1, 3, 1, 0, 0, 0, ..."
...,...,...,...
95,SUPERMARKET_3_499,"[3.044222222222223, 5.732670634920635, 6.45507...","[6, 0, 0, 0, 6, 6, 3, 9, 15, 12, 2, 0, 7, 3, 2..."
96,ACCESORIES_1_158,"[1.1000865800865798, 1.8136150793650792, 1.063...","[1, 1, 0, 0, 0, 1, 2, 1, 2, 0, 0, 0, 0, 1, 2, ..."
97,SUPERMARKET_3_282,"[1.859821428571429, 3.0035699855699853, 8.0113...","[0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 0, 0, 2, 4, 8, ..."
98,ACCESORIES_1_354,"[3.314698412698413, 2.63379761904762, 2.540083...","[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, ..."


## Predigo 30 dias(los de test) 